In [140]:
import pandas as pd
import numpy as np

In [141]:
df=pd.read_csv("/Creditcard_data.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [142]:
y=df['Class']
x=df.drop('Class',axis=1)

In [143]:
y.value_counts()

0    763
1      9
Name: Class, dtype: int64

In [144]:
from imblearn.over_sampling import RandomOverSampler

In [145]:
t=RandomOverSampler(random_state=12)
x_balanced, y_balanced=t.fit_resample(x,y)

In [146]:
y_balanced.value_counts()

0    763
1    763
Name: Class, dtype: int64

SIMPLE RANDOM SAMPLING

In [147]:
x_sample1=x_balanced.sample(300,random_state=12)
y_sample1=y_balanced.sample(300,random_state=12)

SYSTEMATIC SAMPLING

In [148]:
import math
k=math.ceil(len(x_balanced)/300)
x_sample2=x_balanced.iloc[0::k]
y_sample2=y_balanced.iloc[0::k]

STRATIFIED SAMPLING

In [149]:
x_sample3_0=(x_balanced[y_balanced==0]).sample(150)
x_sample3_1=(x_balanced[y_balanced==1]).sample(150)
x_sample3=pd.concat([x_sample3_0,x_sample3_1])

In [150]:
y_sample3_0=y_balanced[y_balanced==0].sample(150)
y_sample3_1=y_balanced[y_balanced==1].sample(150)
y_sample3=pd.concat([y_sample3_0,y_sample3_1])

CLUSTER SAMPLING

In [151]:
def get_clustered_Sample(df, n_per_cluster, num_select_clusters):
    N = len(df)
    K = int(N/n_per_cluster)
    data = None
    for k in range(K):
        sample_k = df.sample(n_per_cluster)
        sample_k["cluster"] = np.repeat(k,len(sample_k))
        df = df.drop(index = sample_k.index)
        data = pd.concat([data,sample_k],axis = 0)

    random_chosen_clusters = np.random.randint(0,K,size = num_select_clusters)
    samples = data[data.cluster.isin(random_chosen_clusters)]
    return(samples)

In [152]:
df_temp=get_clustered_Sample(pd.concat([x_balanced,y_balanced],axis=1),100,3)
df_temp=df_temp.drop('cluster',axis=1)

In [153]:
x_sample4=df_temp.drop('Class',axis=1)
y_sample4=df_temp['Class']

MODEL TRAINING

In [154]:
x_sample=[x_sample1,x_sample2,x_sample3,x_sample4]
y_sample=[y_sample1,y_sample2,y_sample3,y_sample4]

In [155]:
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [156]:
models=['logistic','random_forest','svm','knn','xgboost']
sampling_tech=['simple random','systematic','stratified','cluster']
acc_mat=pd.DataFrame(columns = sampling_tech)

In [157]:
counter = 0
for model in models:
    if model == 'xgboost':
        mod = XGBClassifier()
    elif model == 'logistic':
        mod = LogisticRegression(max_iter=2000)
    elif model == 'random_forest':
        mod = RandomForestClassifier(n_estimators=3, max_depth=4)
    elif model == 'svm':
        mod = SVC()
    elif model == 'knn':
        mod = KNeighborsClassifier(n_neighbors=4)
    for i in range(4):
        X_train, X_test, y_train, y_test = train_test_split(x_sample[i],y_sample[i],test_size=0.3,random_state=42)
        mod.fit(X_train,y_train)
        y_predict = mod.predict(X_test)
        acc_mat.at[counter,str(sampling_tech[i])] = accuracy_score(y_predict, y_test)
    
    counter += 1

In [158]:
acc_mat.index=models
acc_mat

,simple random,systematic,stratified,cluster
logistic,0.888889,0.857143,0.911111,0.9
random_forest,0.988889,0.948052,1.0,0.966667
svm,0.644444,0.584416,0.744444,0.7
knn,0.955556,0.935065,0.9,0.977778
xgboost,0.977778,0.948052,0.955556,0.977778
